# Example Notebook for Text Generation Metric Evaluation

This notebook demonstrates examples use cases for the Valor text generation task type. The Valor text generation task type can be used across a variety of tasks which typically, but not always, involve prompting an LLM to generate some text. Use cases include Query Answering, Retrieval Augmented Generation (which can be thought of as a subcase of Q&A), Summarization and Content Generation. Not all of the text generation metrics make sense for all of these use cases, however these use cases share many common metrics. For example, the BLEU metric can be used to compare to groundtruth answers in the case of Q&A/RAG, and can also be used to compare to groundtruth summaries in the case of Summarization. For another example, both the Coherence and Naturalness metrics can be used to evaluate the quality of generated text across all of these use cases. Some of these use cases also have use case specific metrics, such as ContextRecall for RAG or the Summarization score for Summarization. 

The first example is RAG for Q&A. We get a RAG dataset from HuggingFace, use Llama-Index and GPT3.5 to generate answers, and evaluate those answers with text comparison metrics, RAG metrics and general text generation metrics.

The second example summarization. (TODO get some actual data). Then we evaluate those summaries with text comparison metrics, summarization metrics and general text generation metrics.

The third example is content generation. (TODO get some actual content generation data). Then we evaluate the generated content with general text generation metrics. 

# Use Case #1: RAG for Q&A

## Download and Save the Corpus for the RAG Pipeline

In [ ]:
from datasets import load_dataset

In [ ]:
corpus_dataset = load_dataset("rag-datasets/mini_wikipedia", "text-corpus")["passages"]
print(corpus_dataset)

# For each passage in corpus_dataset, save that passage to a .txt file with the passage_id as the filename.
for passage in corpus_dataset:
    with open(f'./rag_corpus/{passage["id"]}.txt', 'w') as f:
        f.write(passage['passage'])

## Load Queries and get Answers with Llama-Index

In [ ]:
import csv
import os
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

In [ ]:
# Load the query dataset. 
qa_dataset = load_dataset("rag-datasets/mini_wikipedia", "question-answer")["test"]
qa_dataset = qa_dataset.shuffle(seed=42)
print(qa_dataset)

In [ ]:
# Loads in the rag_corpus and builds an index.
# Initially a query_engine, which will use GPT3.5 by default with calls to OpenAI's API.
# You must specify your OpenAI API key in the environment variable OPENAI_API_KEY for the below code to function. 
documents = SimpleDirectoryReader("rag_corpus").load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

In [ ]:
# sample use
response = query_engine.query("What country borders Argentina and Brazil?")
print(response)
print(response.source_nodes)

response = query_engine.query("What color is a penguin?")
print(response)
print(response.source_nodes)

In [ ]:
if os.path.exists('rag_data.csv'):
    os.remove('rag_data.csv')

NUMBER_OF_RECORDS = 50

with open('rag_data.csv', mode='w') as data_file:
    data_writer = csv.writer(data_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    data_writer.writerow(['query', 'groundtruth', 'prediction', 'context_list'])

    for i in range(NUMBER_OF_RECORDS):
        query = qa_dataset[i]['question']
        groundtruth = qa_dataset[i]['answer']
        print(f"{i}: {query}")

        response_object = query_engine.query(query)
        response = response_object.response
        print(f"response: {response}")
        context_list = []
        for i in range(len(response_object.source_nodes)):
            context_list.append(response_object.source_nodes[i].text)
        data_writer.writerow([query, groundtruth, response, context_list])
    
    data_file.close()

## Evaluation in Valor

In this example, the RAG pipeline produces answers to the given queries by retrieving context and then generating answers based on the context and query. Groundtruth answers are also known for these queries. Both the datums (which contain the queries) and the groundtruths are added to the dataset. Then, the predictions are added to the model, which includes the answer and the context used to generate that answer. 

The metrics requested include some text comparison metrics (BLEU, ROUGE1, LDistance), which do a text comparison between the generated answer and the groundtruth answer for the same datum. If the user only desires these metrics, then they do not need to include the context_list in the prediction and they do not need to supply the llm related parameters. 

However, other metrics are requested that use llm guided evaluation (Coherence, ContextRelevance, AnswerRelevance, Hallucination, Toxicity). To get these metrics, the user needs to specify an api url, an api key and a model name, along with any other model kwargs. Each of these metrics will use API calls to the specified LLM service to get information relevant for computing the desired metrics. Some of these metrics, such as Toxicity, do not require any context, so can be used with a Q&A model that does not use context. However other metrics, such as ContextRelevance, require context to be passed in the prediction, as that context is used for computing the metric. 

In [ ]:
import pandas as pd
from valor.enums import TaskType, EvaluationStatus
from valor import Annotation, Datum, Dataset, Model, GroundTruth, Label, Client, Prediction, viz, connect

# Connect to Valor API.
connect("http://0.0.0.0:8000")
client = Client()

In [ ]:
# Read in the dataset of queries, groundtruths and predictions. 
df = pd.read_csv('rag_data.csv')

### Version with Labels and TEXT_GENERATION task type - my current preferred implementation
It seems undesirable to use labels when the labels aren't really doing anything, however the label_ids are useful as identifiers for the resulting metrics. The labels also make it so we don't have to add a bunch of use case specific attributes for each use case. 

In [ ]:
dataset = Dataset.create('rag_dataset')
model = Model.create('rag_model')

for i in range(len(df)):
    row = df.iloc[i]

    # All queries are added to the dataset as Datum objects. 
    datum = Datum(
        uid=i,
        text=row['query'],
    )
    dataset.add_datum(datum)

    # Suppose that only the first half of the queries have groundtruth answers. 
    # The text comparison metrics can only be computed for queries that have groundtruths. 
    if i < len(df)/2:
        dataset.add_groundtruth(
            GroundTruth(
                datum=datum,
                annotations=[
                    # Perhaps you have multiple correct or good groundtruth answers to the query.
                    # The labels below are a trivial example, but you could have less trivial examples.
                    # For example, to the query "When was the United States of America founded?", you might 
                    # consider both "During the American Revolution" or "July 4th, 1776" to be good answers.
                    Annotation(
                        task_type=TaskType.TEXT_GENERATION,
                        labels=[
                            Label(key="answer", value=row['groundtruth']),
                            Label(key="answer", value="The answer is " + row['groundtruth']),
                        ],
                    ),
                ],
            )
        )

dataset.finalize()

for i in range(len(df)):
    row = df.iloc[i]
    datum = Datum(
        uid=i,
        text=row['query'],
    )
    model.add_prediction(
        dataset, 
        Prediction(
            datum=datum,
            annotations=[
                Annotation(
                    task_type=TaskType.TEXT_GENERATION,
                    labels=[Label(key="answer", value=row['prediction'])],
                    context_list=row['context_list'],
                )
            ],
        )
    )

model.finalize_inferences(dataset)

eval_job = model.evaluate_text_generation(
    dataset,
    metrics_to_return=["BLEU", "ROUGE1", 'LDistance', 'Coherence', 'ContextRelevance', 'AnswerRelevance', 'Hallucination', 'Toxicity'],
    llm_model='gpt-3.5-turbo',
    api_url="https://api.openai.com/v1/chat/completions", 
    # api_key=None, # If no key is specified, uses OPENAI_API_KEY or LLM_API_KEY environment variable
    # llm_model_kwargs=None,
)

assert eval_job.wait_for_completion(timeout=30) == EvaluationStatus.DONE

eval_job.metrics

# What might the expected metrics look like? 
# BLEU metric:
# {
#    "value": <BLEU_SCORE>,
#    "label_id": <PREDICTION_LABEL_ID>,
#    "type": "BLEU",
#    "evaluation_id": <EVALUATION_ID>,
# }
# Coherence metric:
# {
#    "value": <COHERENCE_VALUE>,
#    "label_id": <PREDICTION_LABEL_ID>,
#    "type": "Coherence",
#    "evaluation_id": <EVALUATION_ID>,
# } 
# ContextRelevance metric:
# {
#    "value": <CONTEXT_RELEVANCE_VALUE>,
#    "label_id": <PREDICTION_LABEL_ID>,
#    "type": "ContextRelevance",
#    "evaluation_id": <EVALUATION_ID>,
# } 

How it would look if we had use case specific evaluation functions.

In [ ]:
eval_job_text_comp = model.evaluate_text_comparison(
    dataset,
    metrics_to_return=["BLEU", "ROUGE1", 'LDistance'],
    # metric_kwargs=None,
)
assert eval_job_text_comp.wait_for_completion(timeout=30) == EvaluationStatus.DONE
eval_job_text_comp.metrics
# BLEU metric:
# {
#    "value": <BLEU_SCORE>,
#    "label_id": <PREDICTION_LABEL_ID>,
#    "type": "BLEU",
#    "evaluation_id": <EVALUATION_ID>,
# }


eval_job_text_gen = model.evaluate_text_generation( # or evaluate_llm_guided_metrics?
    dataset,
    metrics_to_return=['Coherence', 'Hallucination', 'Toxicity'],
    llm_model='gpt-3.5-turbo',
    api_url="https://api.openai.com/v1/chat/completions", 
    # api_key=None, # If no key is specified, uses OPENAI_API_KEY or LLM_API_KEY environment variable
    # llm_model_kwargs=None,
)
assert eval_job_text_gen.wait_for_completion(timeout=30) == EvaluationStatus.DONE
eval_job_text_gen.metrics
# Coherence metric:
# {
#    "value": <COHERENCE_VALUE>,
#    "label_id": <PREDICTION_LABEL_ID>,
#    "type": "Coherence",
#    "evaluation_id": <EVALUATION_ID>,
# } 


eval_job_llm_guided = model.evaluate_rag(
    dataset,
    metrics_to_return=['ContextRelevance', 'AnswerRelevance'],
    llm_model='gpt-3.5-turbo',
    api_url="https://api.openai.com/v1/chat/completions", 
    # api_key=None, # If no key is specified, uses OPENAI_API_KEY or LLM_API_KEY environment variable
    # llm_model_kwargs=None,
)
assert eval_job_llm_guided.wait_for_completion(timeout=30) == EvaluationStatus.DONE
eval_job_llm_guided.metrics
# ContextRelevance metric:
# {
#    "value": <CONTEXT_RELEVANCE_VALUE>,
#    "label_id": <PREDICTION_LABEL_ID>,
#    "type": "ContextRelevance",
#    "evaluation_id": <EVALUATION_ID>,
# } 

### Version without Labels and QUERY_ANSWERING task type
Potentially looks better to the user but it's awkward to put the query and the answer in the metric. In this implementation, each of the use cases QUERY_ANSWERING, SUMMARIZATION, CONTENT_GENERATION would have their own task type, as each will use different datum and annotation attributes. I don't think this is strictly necessary, but thought that it might make more sense this way if we are adding a bunch of attributes.

In [ ]:
dataset = Dataset.create('rag_dataset')
model = Model.create('rag_model')

for i in range(len(df)/2):
    row = df.iloc[i]

    # All queries are added to the dataset as Datum objects. 
    datum = Datum(
        uid=i,
        query=row['query'],
    )
    dataset.add_datum(datum)

    # Suppose that only the first half of the queries have groundtruth answers. 
    # Some of the below metrics can only be computed for datums that have groundtruths.
    if i < len(df)/2:
        dataset.add_groundtruth(
            GroundTruth(
                datum=datum,
                annotations=[
                    # Perhaps you have multiple correct or good groundtruth answers to the query.
                    # The labels below are a trivial example, but you could have less trivial examples.
                    # For example, to the query "When was the United States of America founded?", you might 
                    # consider both "During the American Revolution" or "July 4th, 1776" to be good answers.
                    # For another example, suppose we have a RAG pipeline
                    Annotation(
                        task_type=TaskType.QUERY_ANSWERING,
                        answer=row['groundtruth'],
                    ),
                    Annotation(
                        task_type=TaskType.QUERY_ANSWERING,
                        answer="The answer is " + row['groundtruth'],
                    ),
                ],
            )
        )

dataset.finalize()

for i in range(len(df)):
    row = df.iloc[i]
    datum = Datum(
        uid=i,
        query=row['query'],
    )
    model.add_prediction(
        dataset, 
        Prediction(
            datum=datum,
            annotations=[
                Annotation(
                    task_type=TaskType.QUERY_ANSWERING,
                    answer=row['prediction'],
                    context_list=row['context_list'],
                )
            ],
        )
    )

model.finalize_inferences(dataset)

eval_job = model.evaluate_query_answering(
    dataset,
    metrics_to_return=["BLEU", "ROUGE1", 'LDistance', 'Coherence', 'ContextRelevance', 'AnswerRelevance', 'Hallucination', 'Toxicity'],
    llm_model='gpt-3.5-turbo',
    api_url="https://api.openai.com/v1/chat/completions", 
    # api_key=None, # If no key is specified, uses OPENAI_API_KEY or LLM_API_KEY environment variable
    # llm_model_kwargs=None,
)

assert eval_job.wait_for_completion(timeout=30) == EvaluationStatus.DONE

eval_job.metrics

# What might the expected metrics look like? 
# BLEU metric:
# {
#    "value": <BLEU_SCORE>,
#    "type": "BLEU",
#    "evaluation_id": <EVALUATION_ID>,
#    "params": {
#         "query": <QUERY>,
#         "answer": <ANSWER>,
#    }
# }
# Coherence metric:
# {
#    "value": <COHERENCE_VALUE>,
#    "type": "Coherence",
#    "evaluation_id": <EVALUATION_ID>,
#    "params": {
#        "query": <QUERY>,
#        "answer": <ANSWER>,
#    }
# } 
# ContextRelevance metric:
# {
#    "value": <CONTEXT_RELEVANCE_VALUE>,
#    "type": "ContextRelevance",
#    "evaluation_id": <EVALUATION_ID>,
#    "params": {
#        "query": <QUERY>,
#        "answer": <ANSWER>,
#    }
# } 

# Use Case #2: Summarization

## Evaluation in Valor

### Version with Labels and TEXT_GENERATION task type - my current preferred implementation

In [ ]:
dataset = Dataset.create('summarization_dataset')
model = Model.create('summarization_model')

for i in range(len(df)):
    row = df.iloc[i]
    datum = Datum(
        uid=i,
        text=row['article'],
    )
    dataset.add_datum(datum)
    dataset.add_groundtruth(
        GroundTruth(
            datum=datum,
            annotations=[
                Annotation(
                    task_type=TaskType.TEXT_GENERATION,
                    labels=[Label(key="summary", value=row['groundtruth'])],
                ),
            ],
        )
    )

dataset.finalize()

for i in range(len(df)):
    row = df.iloc[i]
    datum = Datum(
        uid=i,
        text=row['article'],
    )
    model.add_prediction(
        dataset, 
        Prediction(
            datum=datum,
            annotations=[
                Annotation(
                    task_type=TaskType.TEXT_GENERATION,
                    labels=[Label(key="summary", value=row['prediction'])],
                )
            ],
        )
    )

model.finalize_inferences(dataset)

eval_job = model.evaluate_text_generation(
    dataset,
    metrics_to_return=["BLEU", "ROUGE1", 'LDistance', 'Toxicity', 'Summarization'],
    llm_model='gpt-3.5-turbo',
    api_url="https://api.openai.com/v1/chat/completions", 
    # api_key=None, # If no key is specified, uses OPENAI_API_KEY or LLM_API_KEY environment variable
    # llm_model_kwargs=None,
)

assert eval_job.wait_for_completion(timeout=30) == EvaluationStatus.DONE

eval_job.metrics

# What might the expected metrics look like? 
# BLEU metric:
# {
#    "value": <BLEU_SCORE>,
#    "label_id": <PREDICTION_LABEL_ID>,
#    "type": "BLEU",
#    "evaluation_id": <EVALUATION_ID>,
# }
# Toxicity metric:
# {
#    "value": <TOXICITY_VALUE>,
#    "label_id": <PREDICTION_LABEL_ID>,
#    "type": "Toxicity",
#    "evaluation_id": <EVALUATION_ID>,
# } 
# Summarization metric:
# {
#    "value": <SUMMARIZATION_SCORE>,
#    "label_id": <PREDICTION_LABEL_ID>,
#    "type": "Summarization",
#    "evaluation_id": <EVALUATION_ID>,
# }

How it would look if we had use case specific evaluation functions.

In [ ]:
eval_job_text_comp = model.evaluate_text_comparison(
    dataset,
    metrics_to_return=["BLEU", "ROUGE1", 'LDistance'],
)
assert eval_job_text_comp.wait_for_completion(timeout=30) == EvaluationStatus.DONE
eval_job_text_comp.metrics
# BLEU metric:
# {
#    "value": <BLEU_SCORE>,
#    "label_id": <PREDICTION_LABEL_ID>,
#    "type": "BLEU",
#    "evaluation_id": <EVALUATION_ID>,
# }


eval_job_text_gen = model.evaluate_text_generation( # or evaluate_llm_guided_metrics?
    dataset,
    metrics_to_return=['Toxicity'],
    llm_model='gpt-3.5-turbo',
    api_url="https://api.openai.com/v1/chat/completions", 
    # api_key=None, # If no key is specified, uses OPENAI_API_KEY or LLM_API_KEY environment variable
    # llm_model_kwargs=None,
)
assert eval_job_text_gen.wait_for_completion(timeout=30) == EvaluationStatus.DONE
eval_job_text_gen.metrics
# Toxicity metric:
# {
#    "value": <TOXICITY_VALUE>,
#    "label_id": <PREDICTION_LABEL_ID>,
#    "type": "Toxicity",
#    "evaluation_id": <EVALUATION_ID>,
# } 


eval_job_summ = model.evaluate_summarization(
    dataset,
    metrics_to_return=['Summarization'],
    llm_model='gpt-3.5-turbo',
    api_url="https://api.openai.com/v1/chat/completions", 
    # api_key=None, # If no key is specified, uses OPENAI_API_KEY or LLM_API_KEY environment variable
    # llm_model_kwargs=None,
)
assert eval_job_summ.wait_for_completion(timeout=30) == EvaluationStatus.DONE
eval_job_summ.metrics
# Summarization metric:
# {
#    "value": <SUMMARIZATION_SCORE>,
#    "label_id": <PREDICTION_LABEL_ID>,
#    "type": "Summarization",
#    "evaluation_id": <EVALUATION_ID>,
# }

### Version without Labels and SUMMARIZATION task type

In [ ]:
dataset = Dataset.create('summarization_dataset')
model = Model.create('summarization_model')

for i in range(len(df)):
    row = df.iloc[i]
    datum = Datum(
        uid=i,
        text=row['article'],
    )
    dataset.add_datum(datum)
    dataset.add_groundtruth(
        GroundTruth(
            datum=datum,
            annotations=[
                Annotation(
                    task_type=TaskType.SUMMARIZATION,
                    summary=row['groundtruth'],
                ),
            ],
        )
    )

dataset.finalize()

for i in range(len(df)):
    row = df.iloc[i]
    datum = Datum(
        uid=i,
        text=row['article'],
    )
    model.add_prediction(
        dataset, 
        Prediction(
            datum=datum,
            annotations=[
                Annotation(
                    task_type=TaskType.SUMMARIZATION,
                    summary=row['summary'],
                )
            ],
        )
    )

model.finalize_inferences(dataset)

eval_job = model.evaluate_summarization(
    dataset,
    metrics_to_return=["BLEU", "ROUGE1", 'LDistance', 'Toxicity', 'Summarization'],
    llm_model='gpt-3.5-turbo',
    api_url="https://api.openai.com/v1/chat/completions", 
    # api_key=None, # If no key is specified, uses OPENAI_API_KEY
)

assert eval_job.wait_for_completion(timeout=30) == EvaluationStatus.DONE

eval_job.metrics

# Use Case #3: Content Generation

## Evaluation in Valor

In [ ]:
dataset = Dataset.create('content_generation_dataset')
model = Model.create('content_generation_model')

for i in range(len(df)):
    row = df.iloc[i]
    datum = Datum(
        uid=i,
        text=row['prompt'],
    )
    dataset.add_datum(datum)
    # There are no groundtruths for content generation.

dataset.finalize()

for i in range(len(df)):
    row = df.iloc[i]
    datum = Datum(
        uid=i,
        text=row['prompt'],
    )
    model.add_prediction(
        dataset, 
        Prediction(
            datum=datum,
            annotations=[
                Annotation(
                    task_type=TaskType.TEXT_GENERATION,
                    labels=[Label(key="generated_content", value=row['output'])],
                )
            ],
        )
    )

model.finalize_inferences(dataset)

eval_job = model.evaluate_text_generation(
    dataset,
    metrics_to_return=['Coherence', 'Toxicity', 'Bias'],
    llm_model='gpt-3.5-turbo',
    api_url="https://api.openai.com/v1/chat/completions", 
    # api_key=None, # If no key is specified, uses OPENAI_API_KEY or LLM_API_KEY environment variable
    # llm_model_kwargs=None,
)

assert eval_job.wait_for_completion(timeout=30) == EvaluationStatus.DONE

eval_job.metrics

# What might the expected metrics look like? 
# Toxicity metric:
# {
#    "value": <TOXICITY_VALUE>,
#    "label_id": <PREDICTION_LABEL_ID>,
#    "type": "Toxicity",
#    "evaluation_id": <EVALUATION_ID>,
# } 

If we were to break things into different task types, we might still consider content generation as part of Q&A. Content Generation fits the same format as Q&A, except that a user should not use text comparison metrics to compare generated content to a groundtruth. The only way to prevent the user from uploading groundtruths and using the text comparison method would be to have a separate task type for content generation. But if the user actually has groundtruths that they want to compare to, then why should we bother restricting them in this way? Maybe there is a content generation use case that we are not thinking about right now where a user might actually want to compare generated content to some other text. 